# Video Tag Generation

Generate video tags based on Azure Content Understanding and Azure OpenAI.


## Pre-requisites
1. Follow [README](../docs/create_azure_ai_service.md) to create essential resource that will be used in this sample.
1. Install required packages

In [ ]:
%pip install -r ../requirements.txt

## Load environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)

AZURE_AI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_SERVICE_ENDPOINT")
AZURE_AI_SERVICE_API_VERSION = os.getenv("AZURE_AI_SERVICE_API_VERSION", "2024-12-01-preview")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

AUTHENTICATION_URL = os.getenv("AUTHENTICATION_URL")

## File to Analyze

In [ ]:
from pathlib import Path
VIDEO_LOCATION = Path("../data/FlightSimulator.mp4")

## Create a custom analyzer and submit the video to generate tags
The custom analyzer schema is defined in **../analyzer_templates/video_tag.json**. The custom fields are `segmentDescription`, `transcript` and `tags`. Adding description and transcripts helps to increase the accuracy of tag generation results. To get transcripts, we will need to set the`returnDetails` parameter in the `config` field to `True`.

In this example, we will use the utility class `AzureContentUnderstandingClient` to load the analyzer schema from the template file and submit it to Azure Content Understanding service. Then, we will analyze the video to get the segment tags.


In [ ]:
import sys
from pathlib import Path
import json
import uuid


# add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(
    str(parent_dir)
)
from python.content_understanding_client import AzureContentUnderstandingClient

from azure.identity import AzureCliCredential, get_bearer_token_provider
credential = AzureCliCredential()
token_provider = get_bearer_token_provider(credential, AUTHENTICATION_URL)

# The analyzer template is used to define the schema of the output
ANALYZER_TEMPLATE_PATH = "../analyzer_templates/video_tag.json"
ANALYZER_ID = "video_tag" + "_" + str(uuid.uuid4())  # Unique identifier for the analyzer

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    token_provider=token_provider,
#    x_ms_useragent="azure-ai-tag-with-content-understanding-python/video_tag", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(
    ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE_PATH)
result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content
It might take some time depending on the video length. Try with short videos to get results faster

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=VIDEO_LOCATION)

# Wait for the analysis to complete and get the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout for long videos

# Print the content analysis result
print(f"Video Content Understanding result: ", video_cu_result)

# Optional - Delete the analyzer if it is no longer needed
cu_client.delete_analyzer(ANALYZER_ID)

## Aggregate tags from each segment to generate video tags

ChatGPT will be used to remove duplicate tags which are semantically similar across segments.



In [ ]:
from python.utility import OpenAIAssistant, aggregate_tags

# Create an OpenAI Assistant to interact with Azure OpenAI
openai_assistant = OpenAIAssistant(
    aoai_end_point=AZURE_OPENAI_ENDPOINT,
    aoai_api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
    aoai_api_key=None,
)

# Aggregate tags using the video segment result from Azure Content Understanding
tag_result = aggregate_tags(video_cu_result, openai_assistant)

tag_result.tags